In [1]:
from subprocess import Popen, PIPE, STDOUT
from random import randint
import binascii
import os
from queue import Queue, Empty
from threading  import Thread
from time import sleep
import random
import nltk
nltk.download('averaged_perceptron_tagger')
from itertools import permutations, combinations
import spacy
from spacy.matcher import Matcher
from spacy.attrs import POS
nlp = spacy.load('en_core_web_sm')
import pandas as pd
nltk.download('punkt')
import numpy as np
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import timeit
from keras.models import Model
from keras.layers import *   
from progressbar import * 
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
from keras.preprocessing.sequence import pad_sequences
from agent import DQNAgent
import importlib
from collections import deque

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [330]:
basic_actions = [
           'go north',
           'go south',
           'go west',
           'go east',
           'go northeast',
           'go northwest',
           'go southeast',
           'go southwest',
           'd',
           'u'
]

directions = ['north', 'south', 'east', 'west', 'northwest', 'northeast', 'southwest', 'southeast']

action_space = set()

for action in basic_actions:
    action_space.add(action)

command1_actions = ['open OBJ',
 'get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'ask OBJ',
 'fix OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'go OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'burn OBJ',
 'inflate OBJ',
 'brandish OBJ',
 'donate OBJ',
 'squeeze OBJ',
 'attach OBJ',
 'find OBJ',
 'banish OBJ',
 'read OBJ',
 'enchant OBJ',
 'feel OBJ',
 'pour OBJ']
                  

command2_actions = ['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeeze OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandish OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflate OBJ with DCT',
 'unlock OBJ with DCT',
 'give OBJ to DCT', 
 'carry OBJ to DCT',
 'spray OBJ with DCT',

]
                    

filtered_tokens = ['Score', 'Moves']

In [3]:
filtered_tokens

['Score', 'Moves']

In [4]:
def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

In [5]:
def start_game():
    score = 0
    moves = 0
    p = Popen(['dfrotz.exe','zork1.z5'], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    q = Queue()
    t = Thread(target=enqueue_output, args=(p.stdout, q))
    t.daemon = True # thread dies with the program
    t.start()
    sleep(0.1)
    return(p, q, t, score, moves)

In [247]:
# read line without blocking
def readLine(q):
    cont = True
    narrative = ""
    while cont:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            cont = False
        else: 
            narrative = narrative + line.decode("utf-8").replace('\n', " ").replace('\r', " ")
    if ('840726' in narrative): ## Opening narrative
        narrative = narrative[narrative.index('840726') + len('840726'):]
    try:
        score, moves = grab_score_moves(narrative)
        narrative = narrative[narrative.index('Moves: ')+len('Moves:')+5:-1].strip()
    except:  ## not valid move
        pass
    sleep(0.1)
    return(narrative, score, moves)

In [7]:
def look_surroundings(p):
    perform_action('look', p)

In [8]:
def check_inventory(p):
    perform_action('inventory', p)

In [9]:
def grab_score_moves(narrative):
    try:
        score = int(narrative[narrative.index('Score: ') + len('Score: '):][0:3].strip())
        moves = int(narrative[narrative.index('Moves: ') + len('Moves: '):][0:3].strip())
    except:  ## not valid move
        score = 0
        moves = 0
    return(score, moves)

In [10]:
def detect_nouns(narrative):  #grabs nouns and adjective noun pairs from narrative
    
    processed = preprocess(narrative)
    pattern = 'NP: {<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(processed)
    noun_list =  [noun.text for noun in nlp(narrative).noun_chunks]
    for direction in directions:
        if direction in noun_list:
            noun_list.remove(direction)
    return(noun_list)
    
    # function to test if something is a noun
    #is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    #tokenized = nltk.word_tokenize(narrative)
    #nouns = [word.lower() for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    #return(nouns)

In [239]:
def get_nouns(narrative):
    matcher = Matcher(nlp.vocab)
    matcher.add('Noun phrase', None, [{POS: 'NOUN'}])
    doc = nlp(narrative)
    matches = matcher(doc)
    noun_list = [doc[start:end].text for id, start, end in matches]
    for direction in directions:
        if direction in noun_list:
            noun_list.remove(direction)
    for invalid in invalid_nouns:
        if invalid in noun_list:
            noun_list.remove(invalid)
    return(noun_list)

In [12]:
def generate_action_tuples(nouns):
    possible_actions = []
    similarities = []
    for i in basic_actions:
        possible_actions.append(i)
        similarities.append(.5)
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('OBJ', i)
            possible_actions.append(action_to_add)
            try:
                similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))
            except:
                similarities.append(0.02)
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('OBJ', perm[0])
                    action_to_add = action_to_add.replace('DCT', perm[1])
                    possible_actions.append(action_to_add)
                    try:
                        similarities.append(model.similarity(word_tokenize(action_to_add)[0], i))
                    except:
                        similarities.append(0.02)

    return possible_actions

In [13]:
from numpy.random import choice
def selectOne(action_space, similarities):
    return action_space[choice(len(action_space), p=similarities)]

In [14]:
import math
def add_to_action_space(action_space, actions):
    similarities = []
    
    for action in actions:
        action_space.add(action)
    for action in action_space:
        words = word_tokenize(action)
        verb = words[0]
        if verb in basic_actions:
            similarities.append(0.5)
        elif len(words)<3:
            noun = word_tokenize(action)[1]
            try:
                sim_score = model.similarity(verb, noun)**4
                if sim_score < 0:
                    sim_score = 0.05**4
                similarities.append(sim_score)
            except:
                similarities.append(0.02**4)
                
        else: 
            try:
                noun1 = word_tokenize(action)[1]
                prep = word_tokenize(action)[2]
                noun2 = word_tokenize(action)[3]
                sim_score1 = model.similarity(verb, noun1)
                sim_score2 = model.similarity(prep, noun2)
                sim_score = ((sim_score1 + sim_score2)/2)**4
                if sim_score < 0:
                    sim_score = 0.05
                similarities.append(sim_score**4)
            except:
                similarities.append(0.02**4)
            

    return action_space, similarities

In [15]:
def perform_action(command, p):
    p.stdin.write(bytes(command+ "\n", 'ascii'))
    p.stdin.flush()
    sleep(0.1)## wait for action to register

In [16]:
def kill_game(p):
    p.terminate()
    p.kill()

In [17]:
def choose_random_action(p, q, action_space):
    action = random.sample(action_space, 1)
    return action[0]

In [18]:
def buildWord2VecModel():
    #model = Word2Vec.load('tutorial.model')
    f = open('tutorials_2.txt', 'r')
    tutorials = f.read()
    sentences = word_tokenize(tutorials)
    model = Word2Vec([sentences])
    return model

In [315]:
def preprocess(text):
    # fix bad newlines (replace with spaces), unify quotes
    text = text.strip()
    text = text.replace('\\n', ' ').replace('‘', '\'').replace('’', '\'').replace('”', '"').replace('“', '"')
    # convert to lowercase
    text = text.lower()
    # remove all characters except alphanum, spaces and - ' "
    text = re.sub('[^ \-\sA-Za-z0-9"\']+', ' ', text)
    # split numbers into digits to avoid infinite vocabulary size if random numbers are present:
    text = re.sub('[0-9]', ' \g<0> ', text)
    # expand unambiguous 'm, 't, 're, ... expressions
    text = text. \
            replace('\'m ', ' am '). \
            replace('\'re ', ' are '). \
            replace('won\'t', 'will not'). \
            replace('n\'t', ' not'). \
            replace('\'ll ', ' will '). \
            replace('\'ve ', ' have '). \
            replace('\'s', ' \'s')
    return text

In [26]:
def init_tokenizer(vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size)
    return tokenizer

In [27]:
def vectorizeText(text, tokenizer):
    text = preprocess(text)
    words = word_tokenize(text)
    tokenizer.fit_on_texts(words)
    seq = tokenizer.texts_to_sequences(words)
    sent = []
    for i in seq:
        sent.append(i[0])
    padded = pad_sequences([sent], maxlen=50, padding='post')
    return (padded)

In [319]:
def calculate_reward(story, moves_count, new_narrative):
    reward = 0
    
    ## add reward from score in game
    if(moves_count != 0):
        new_score = int(story['Score'][moves_count]) - int(story['Score'][moves_count-1])
        reward = reward + new_score*5
    
    ## add small negative reward for each move
    reward = reward - 1
    
    ## add reward for picking up / using items
    if(moves_count != 0):
        pre_inventory = story['Inventory'][moves_count-1]
        inventory = story['Inventory'][moves_count]
        if pre_inventory != inventory:  ## inventory changed
            reward = reward + 20
            print('inventory changed')

            
    ## add reward for discovering new areas
    if new_narrative not in unique_narratives:  ## new location
        reward = reward + 10
        print('discovered new area')

    
    ## detect and remove invalid nouns from future turns
    response = str(story['Response'][moves_count]).lower()
    if('don\'t know the word' in response):
        startIndex = response.find('\"')
        endIndex = response.find('\"', startIndex + 1)
        word = response[startIndex+1:endIndex]
        print('Didn\'t know the word ' + word)
        invalid_nouns.append(word)
    print(reward)
    return reward

In [304]:
model = buildWord2VecModel()

In [320]:
tokenizer = init_tokenizer(500)

In [321]:
import agent

In [322]:
importlib.reload(agent)

<module 'agent' from 'C:\\Users\\sparr\\Desktop\\datascience\\dsc-5-capstone-project-online-ds-sp-000\\agent.py'>

In [323]:
from agent import DQNAgent

In [324]:
agent = DQNAgent()

In [325]:
invalid_nouns = []
unique_narratives = set()
story = pd.DataFrame(columns=['Surroundings', 'Inventory', 'Action', 'Response', 'Score', 'Moves'])
rounds = 400
pbar = ProgressBar(maxval=rounds)
pbar.start()
p, q, t, score, moves = start_game()
moves_count = 0
batch_size = 20
try:
    for i in range(0,rounds):
        ## Check surroundings, check inventory, choose action, check action response
        narrative,score,moves = readLine(q)
        check_inventory(p)
        inventory,s,m = readLine(q)

        nouns = get_nouns(narrative)

        # build action space
        current_action_space = generate_action_tuples(nouns)
        action_space = set()
        action_space, probs = add_to_action_space(action_space, current_action_space)
        actions = []
        for a in action_space:
            actions.append(a)
            
        narrativeVector = vectorizeText(narrative,tokenizer)

            
        if (agent.act_random() or i < 10): ## choose random action
            print('random choice')
            probs = np.array(probs)
            probs /= probs.sum()
            action = selectOne(actions, probs)
            df = pd.DataFrame(columns=['Action', 'Prob'])
            df['Action'] = actions
            df['Prob'] = probs
            
        else: ## choose predicted max Q value action
            print('predicted choice')
            actionsVectors = []
            for a in actions:
                actionsVectors.append(vectorizeText(a,tokenizer))
            best_action, max_q = agent.predict_actions(narrativeVector, actionsVectors)
            action = actions[best_action]
        
        perform_action(action, p)

        response,score,moves = readLine(q)
        story.loc[i] = [narrative, inventory, action, response, str(s), str(i+1)]
        unique_narratives.add(preprocess(narrative))
        
        actionVector = vectorizeText(action,tokenizer)
                
        look_surroundings(p)
        new_narrative,s,m = readLine(q)
        new_narrativeVector = vectorizeText(new_narrative, tokenizer)
        
        
        reward = calculate_reward(story, moves_count, preprocess(new_narrative))
        moves_count += 1

        
        agent.remember(narrativeVector, actionVector, reward, new_narrativeVector, False)
        
        look_surroundings(p)
        
        if moves_count%batch_size == 0:  ## batch of 5 experiences in memory
            agent.replay(batch_size)
    
        pbar.update(i) #this adds a little symbol at each iteration
    pbar.finish()
    kill_game(p)
except Exception as e: 
    kill_game(p)
    print(e.with_traceback())

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if sys.path[0] == '':
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  from ipykernel import kernelapp as app
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similari

random choice
-1
random choice
discovered new area
9
random choice
-1
random choice
Didn't know the word side
-1
random choice
-1


random choice
-1
random choice
-1
random choice
-1
random choice
-1


random choice
-1
random choice
discovered new area
9
random choice
-1
random choice
-1


random choice
Didn't know the word corner
-1
random choice
-1
random choice
-1
random choice
-1


...

random choice
-1
random choice
-1
random choice
-1
random choice
-1


random choice
-1
random choice
-1
random choice
-1
random choice
-1


random choice
-1
random choice
-1
random choice
-1
random choice
-1


random choice
-1
random choice
-1
random choice
-1


100% |########################################################################|


In [326]:
import pickle

with open('invalid_nouns.txt', 'wb') as fp:
    pickle.dump(invalid_nouns, fp)

In [329]:
with open ('invalid_nouns.txt', 'rb') as fp:
    test = pickle.load(fp)
test

['side',
 'corner',
 'canyon',
 'canyon',
 'canyon',
 'cavern',
 'view',
 'top',
 'distance',
 'edge',
 'directions',
 'preparation',
 'room',
 'room',
 'living',
 'battery',
 'center',
 'doorway']

In [327]:
unique_narratives

{'behind house  you are behind the white house  a path leads into the forest to the east   in one corner of the house there is a small window which is open ',
 'behind house  you are behind the white house  a path leads into the forest to the east   in one corner of the house there is a small window which is slightly ajar ',
 'canyon view  you are at the top of the great canyon on its west wall  from here there is  a marvelous view of the canyon and parts of the frigid river upstream   across the canyon  the walls of the white cliffs join the mighty ramparts  of the flathead mountains to the east  following the canyon upstream to the  north  aragain falls may be seen  complete with rainbow  the mighty frigid  river flows out from a great dark cavern  to the west and south can be seen  an immense forest  stretching for miles around  a path leads northwest  it  is possible to climb down into the canyon from here ',
 'clearing  you are in a clearing  with a forest surrounding you on all s

In [334]:
agent.memory

deque([])

In [328]:
story

,Surroundings,Inventory,Action,Response,Score,Moves
0,West of House You are standing in an open...,You are empty-handed.,open door,The door cannot be opened.,0,1
1,West of House You are standing in an open fie...,You are empty-handed.,go north,North of House You are facing the north side ...,0,2
2,North of House You are facing the north side ...,You are empty-handed.,read north,">You used the word ""north"" in a way that I don...",0,3
3,North of House You are facing the north side ...,You are empty-handed.,read side with door,">I don't know the word ""side"".",0,4
4,North of House You are facing the north side ...,You are empty-handed.,fill north,">You used the word ""north"" in a way that I don...",0,5
5,North of House You are facing the north side ...,You are empty-handed.,pick door,You can't see any door here!,0,6
6,North of House You are facing the north side ...,You are empty-handed.,wear north,">You used the word ""north"" in a way that I don...",0,7
7,North of House You are facing the north side ...,You are empty-handed.,put door,>What do you want to put the door in?,0,8
8,North of House You are facing the north side ...,You are empty-handed.,go south,The windows are all boarded.,0,9
9,North of House You are facing the north side ...,You are empty-handed.,pick door with north,">You used the word ""north"" in a way that I don...",0,10


In [335]:
df.sort_values(by='Prob', ascending=False).head(100)

,Action,Prob
376,go south,0.013467
2426,go west,0.013459
1996,go east,0.013450
176,go north,0.013447
1853,go northwest,0.013264
1120,go northeast,0.012702
881,go southeast,0.012380
1783,go southwest,0.012195
504,get passage,0.012023
1574,go passage,0.012014


In [296]:
df.sort_values(by='Prob', ascending=False)

,Action,Prob
228,go south,3.307232e-02
196,go west,3.305153e-02
368,go east,3.302998e-02
125,go north,3.302246e-02
19,open door,3.289401e-02
251,go door,3.288875e-02
351,get door,3.278160e-02
287,go northwest,3.257438e-02
389,read door,3.239859e-02
12,put door,3.203310e-02


In [297]:
df[df.Action == 'open mailbox']

,Action,Prob
61,open mailbox,3.111646e-07


In [298]:
df.Prob.mean()

0.0022624434389140274

In [50]:
model.save('tutorial.model')

In [52]:
model = buildWord2VecModel()

In [55]:
model.similarity('open', 'door')

C:\Users\sparr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.9984921

In [109]:
kill_game(p)
